In [1]:
import os
import re
import pandas as pd
import numpy as np
import tqdm

In [2]:
%pip install openai
# import openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
openai.api_type = "azure"
# openai.azure_endpoint = 
# openai.api_version = "2023-03-15-preview"
# openai.api_key = 


import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key= 
    api_version="2023-10-01-preview",
    azure_endpoint = 
)

In [4]:
def getPrompt(prompt,prefix=None):

    if(prefix is not None):
        default = prefix
    else:
        default = '''Augment the passage as shown by adding one new variable. 
Rules:
1. The new variable must change values over the course of the augmented passage.
2. Old and new value of the new variable should be explicitly mentioned.
3. The new variable must be one of the following types: [Volume, Humidity, Temperature, Weight, Luminosity, Density, Speed, Area]. 
4. The new variable must not be related to or derived from the existing variables in the passage.
5. The new variable must not share the same physical unit as any of the original variables. 
6. The augmented text should not add any numerical information about existing variables if it did not exist in the original passage.
7. Add no more than one sentence.

If you follow all the rules, you will win $200. If even a single rule is broken, the world will end.

Example 1:

Passage:  My car gets 20 miles per gallon.
Existing Variables: Fuel efficiency
Augmented: My car, which gets 20 miles for each gallon, when I drive at 120 miles per hour. I start going at 10 feet a minute. 
New Variables: Speed (changes from 120 to 10)

Explanation: the new variable Speed is added. It is independent of the Fuel efficiency variable in the original passage. Speed cannot be derived from the original passage. The units for speed are not related to the units for fuel efficiency, fulfilling Rule 4. The augmented passage does not add any information about existing variables (fuel efficiency) at all, fulfilling Rule 6. 

Example 2:
Passage:  There are 64 pigs in the barn. Some more came in, now there are 86 pigs.
Existing Variables: Number of pigs (changes from 64 to 86)
Augmented: In the barn, where the temperature is a cozy 72 degrees Fahrenheit, there are 64 pigs. Some more came in. The temperature goes up to 83 degrees fahrenheit for 86 pigs. 
New variables: Temperature (changes from 72 to 83)

Explanation: the new variable Temperature is added that is independent of the number of pigs variable. The units for temperature are not related to the units for the number of pigs, fulfilling Rule 4.  The augmented passage does not add any information about existing variables (number of pigs) at all, fulfilling Rule 6. 

Now output the existing variables and augment the passage below following all rules. Also print the new variables in the augmented passage.
Passage: {}
Existing Variables:'''.format(prompt)


    return default


In [5]:

# def getInference(prompt_text, returnPrompt=False):

#     response = openai.ChatCompletion.create(
#         engine="gpt-35-turbo",
#         messages = [

#                         {"role": "system", "content": "You are a mathematics and linguistics professor setting a test."},

#                         {"role": "user", "content": prompt_text}

#                     ],
#         temperature=0.7,
#         max_tokens=800,
#         top_p=0.95,
#         frequency_penalty=0,
#         presence_penalty=0,
#         stop=None)

#     ans = response["choices"][0]["message"]["content"].split("\n")

#     if(returnPrompt):
#         return [prompt_text,ans]
#     else:
#         return [ans] 



def getInference(prompt_text, returnPrompt=False):

    # print(prompt_text)
    # print(type(prompt_text))

    response = openai.chat.completions.create(
    # response = client.completions.create(
        model="gpt-35-turbo",
        messages = [

                        {"role": "system", "content": "You are a mathematician."},

                        {"role": "user", "content": prompt_text}

                    ],
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)

    ans = response.choices[0].message.content

    if(returnPrompt):
        return [prompt_text,ans[0]]
    else:
        return [ans ]

In [21]:

ipfile = 'stage4_train_processed_data.json'
opfile = 'SIMPLE-stage_4_with_adversarial_TRAIN.csv'

# ipfile = 'stage2_test_with_answer_processed_data.json'
# opfile = 'stage_2_with_adversarial_TEST.csv'

In [22]:
file = pd.read_json(ipfile)
file.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, passage to qa_pairs
Columns: 472 entries, passage_0 to passage_471
dtypes: object(472)
memory usage: 7.4+ KB


In [23]:
passage_ids = file.columns

In [24]:
ip_list2 = passage_ids

count = 0

batch_size = 50  #Tunable parameter


In [25]:
count

0

In [26]:
total_shards = len(ip_list2)/batch_size

In [27]:
total_shards

9.44

In [28]:

while len(ip_list2) > 0:
    temp_list = ip_list2[:batch_size]

    para=[]
    question=[]
    answer=[]
    query_id=[]
    is_adversarial=[]
    passage_ids_L=[]

    print('creating shard ',count)
 

    for p in tqdm.tqdm(temp_list):
        passage_ids_L.append(p)
        passage_ids_L.append(p)

        para.append(file[p]['passage'])
        question.append(file[p]['qa_pairs'][0]['question'])
        answer.append(file[p]['qa_pairs'][0]['answer'])
        query_id.append(file[p]['qa_pairs'][0]['query_id'])
        is_adversarial.append(False)

        para.append(getInference(getPrompt(file[p]['passage'])))
        question.append(file[p]['qa_pairs'][0]['question'])
        answer.append(file[p]['qa_pairs'][0]['answer'])
        query_id.append(file[p]['qa_pairs'][0]['query_id'])
        is_adversarial.append(True)
    
    dff = pd.DataFrame(data={'passage_id':passage_ids_L,
                        'passage':para,
                        'question':question,
                        'answer':answer,
                        'query_id':query_id,
                        'isAdversarial':is_adversarial})

    dff.to_csv(opfile.replace('.csv','_SHARD_{}.csv'.format(count)),index=False)
    
    count+=1
    print("shard number:",count,"Done")
    ip_list2 = ip_list2[batch_size:]

creating shard  0
shard number: 1 Done
creating shard  1
shard number: 2 Done
creating shard  2
shard number: 3 Done
creating shard  3
shard number: 4 Done
creating shard  4
shard number: 5 Done
creating shard  5
shard number: 6 Done
creating shard  6
shard number: 7 Done
creating shard  7
shard number: 8 Done
creating shard  8
shard number: 9 Done
creating shard  9
shard number: 10 Done


100%|██████████| 22/22 [00:39<00:00,  1.82s/it]


In [29]:
count

10

In [30]:
# ip_list = list(df[0])

In [31]:

    # para.append(getInference(getPrompt(file[p]['passage'])))
    # question.append(file[p]['qa_pairs'][0]['question'])
    # answer.append(file[p]['qa_pairs'][0]['answer'])
    # query_id.append(file[p]['qa_pairs'][0]['query_id'])
    # is_adversarial.append(True)

In [32]:
# dff[dff.isAdversarial==True]['passage'].iloc[5]#[0][1].split(':')[-1]

In [33]:
# dff.loc[dff['isAdversarial']==True
#                 , 'para'] = dff.loc[dff['isAdversarial']==True,'para'].str.split('Augmented:').str[-1]


In [34]:
# dff.head()

In [35]:
# file['passage_37']['qa_pairs'][0]